In [4]:
import os
import torch as th
import numpy as np
import gymnasium as gym
import pandas as pd
from stable_baselines3 import DQN
from scripts.utils import load_scenario_data
from src.environment import AircraftDisruptionEnv
from datetime import datetime


# Initialize the environment
def initialize_environment(data_dict):
    aircraft_dict = data_dict['aircraft']
    flights_dict = data_dict['flights']
    rotations_dict = data_dict['rotations']
    alt_aircraft_dict = data_dict['alt_aircraft']
    config_dict = data_dict['config']

    # Check if there are any flights in the flights_dict for the current day
    this_day_flights = [flight_info for flight_info in flights_dict.values() if '+' not in flight_info['DepTime']]
    
    if not this_day_flights:
        # Handle empty flight list
        raise ValueError(f"No valid flights found in the scenario {data_dict['scenario_name']}. Skipping this scenario.")
    
    # Initialize the environment
    env = AircraftDisruptionEnv(
        aircraft_dict, 
        flights_dict, 
        rotations_dict, 
        alt_aircraft_dict, 
        config_dict
    )
    
    return env

# Load the model and evaluate on all scenarios in the folder
def evaluate_model_on_scenarios(model_path, scenarios_folder, output_csv):
    # Verify that the model exists
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model file not found: {model_path}")

    # Initialize a list to store rewards for all scenarios
    results = []

    # Load the trained model
    model = DQN.load(model_path)

    # Iterate over all scenario folders in the specified directory
    scenario_folders = [os.path.join(scenarios_folder, folder) for folder in os.listdir(scenarios_folder)
                        if os.path.isdir(os.path.join(scenarios_folder, folder))]

    for scenario_folder in scenario_folders:
        try:
            # print(f"Evaluating scenario: {scenario_folder}")
            
            # Load the scenario data
            data_dict = load_scenario_data(scenario_folder)
            data_dict['scenario_name'] = os.path.basename(scenario_folder)  # Add scenario name for error handling
            
            # Initialize the environment with validation for empty flights
            env = initialize_environment(data_dict)

            # Set the environment for the model
            model.set_env(env)

            # Reset the environment
            obs, _ = env.reset()
            done_flag = False
            total_reward = 0
            step_num = 0
            max_steps = 1000  # Set a maximum number of steps to prevent infinite loops

            # Run the environment loop
            while not done_flag and step_num < max_steps:
                # Predict the action using the trained model
                action, _ = model.predict(obs, deterministic=True)
                obs, reward, terminated, truncated, _ = env.step(action)

                # Accumulate the reward
                total_reward += reward

                # Combine terminated and truncated flags
                done_flag = terminated or truncated
                step_num += 1
            
            # Append the results (scenario name, total reward, and steps)
            results.append({
                "scenario": os.path.basename(scenario_folder),
                "total_reward": total_reward,
                "steps": step_num
            })

        except ValueError as ve:
            print(f"Error in scenario {scenario_folder}: {ve}")
            continue

    # Convert results to DataFrame and save to CSV
    df = pd.DataFrame(results)
    os.makedirs(os.path.dirname(output_csv), exist_ok=True)
    df.to_csv(output_csv, index=False)

    print(f"Results saved to: {output_csv}")

# Specify the model path and scenarios folder
MODEL_PATH = "../trained_models/dqn_100000d_1000u-1.zip"
SCENARIOS_FOLDER = "../data/Testing/1k-3ac-12f-1dis-F/"
OUTPUT_CSV = "../results/evaluations/evaluation_results.csv"

# Run the evaluation on all scenarios
evaluate_model_on_scenarios(MODEL_PATH, SCENARIOS_FOLDER, OUTPUT_CSV)



Error in scenario ../data/Testing/1k-3ac-12f-1dis-F/Scenario_201: No valid flights found in the scenario Scenario_201. Skipping this scenario.
Results saved to: ../results/evaluations/evaluation_results.csv
